In [15]:
#Front End Portal for Remote Sensing data

#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sympy.abc import m,v
import geemap
import ipywidgets as widgets 
import ipyleaflet
from ipywidgets import AppLayout
from IPython.display import display, clear_output, HTML
import ee
import cv2
import joblib

In [16]:
#mask clouds in the satellite images
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
               qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000) 

In [17]:
#Portal design
#Map
mapWidget = widgets.Output()
    
#labels
step1 = widgets.Label('Step 1: Select the date')
step2 = widgets.Label('Step 2: Select Region of Interest')
step3 = widgets.Label('Step 3: Plastic Detection')
step4 = widgets.Label('Step 4: Display Output')

In [18]:
#buttons
#date
dateBox = widgets.DatePicker(
    description = '',
    disabled = False
)

#ROI button
getROI = widgets.Button(description='OK')
#detect plastic button
estimate_plastics_Btn = widgets.Button(description='Detect Plastic')

#output widgets
estimate_plastics_plot = widgets.Output()
out1 = widgets.Output()
out2 = widgets.Output()

In [20]:
#collect satellite images based on date and ROI
def getStatistics(change):
    
    global B8_image, B4_image, B3_image, B6_image, B11_image, B8_Matrix, B4_Matrix, B3_Matrix, B6_Matrix, B11_Matrix
    
    with out1:
        clear_output()
        print("Loading Data...")
        
    #get the ROI stats
    feature = Map.draw_last_feature
    roi = feature.geometry()

    #get area of the ROI in sq.meter
    area = roi.area(1).getInfo()   
    
    #get the year, month, day from the user selected date
    year = dateBox.value.year
    month = dateBox.value.month
    day = dateBox.value.day
    
    #format date
    strt_date = str(year) + "-" + str(month) + "-" + str(day)
    end_date = pd.Timestamp(strt_date) + pd.DateOffset(days = 5)
    end_date = end_date.date()
    
    try:
        # Load Sentinel-2 TOA reflectance data from GEE
        imageCollection = ee.ImageCollection('COPERNICUS/S2') \
                            .filterDate(strt_date, str(end_date)) \
                            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
                            .map(maskS2clouds) \
                            .filterBounds(roi) \
                            .sort('CLOUDY_PIXEL_PERCENTAGE')

        #get the number of images collected over the time period
        count = imageCollection.size()
        #get the first image in the collection
        rawImg = ee.Image(imageCollection.toList(imageCollection.size()).get(0))   

        #get the bands data
        B8_image = geemap.ee_to_numpy(rawImg.select(['B8']), region=roi)
        B4_image = geemap.ee_to_numpy(rawImg.select(['B4']), region=roi)
        B3_image = geemap.ee_to_numpy(rawImg.select(['B3']), region=roi)
        B6_image = geemap.ee_to_numpy(rawImg.select(['B6']), region=roi)
        B11_image = geemap.ee_to_numpy(rawImg.select(['B11']), region=roi)
    
    
        #set bands to same resolution
        targetSize = (40,40)
        B8_Matrix = cv2.resize(B8_image,targetSize,interpolation=cv2.INTER_NEAREST)
        B4_Matrix = cv2.resize(B4_image,targetSize,interpolation=cv2.INTER_NEAREST)
        B3_Matrix = cv2.resize(B3_image,targetSize,interpolation=cv2.INTER_NEAREST)
        B6_Matrix = cv2.resize(B6_image,targetSize,interpolation=cv2.INTER_NEAREST)
        B11_Matrix = cv2.resize(B11_image,targetSize,interpolation=cv2.INTER_NEAREST)

        #get date of the accquired image
        img_id = rawImg.get('system:index').getInfo()
        rawImg1 = ee.Image('COPERNICUS/S2/{}'.format(img_id))
        ee_date = ee.Date(rawImg1.get('system:time_start'))
        cloud_cover = rawImg1.get("CLOUD_COVERAGE_ASSESSMENT")
    
        #display the image metadata
        #with roi_out:
        with out1:
            clear_output()
            print("Image Metadata:")
            print("Accquistion Date:",ee_date.format().getInfo())
            print("Cloud Cover pixel: {}%".format(round(cloud_cover.getInfo(),2)))
            #if count == 1:
                #   print("Count:",count.getInfo())
            print("ROI Area: {0} {1}".format(round(area,2),"m\u00b2"))

            #plot raw satellite image
            plt.imshow(B4_Matrix)
            plt.title("Raw Satellite Image")
            plt.axis("off")
            plt.show()    
    except:
         with out1:
            clear_output()
            print("No satellite image found on the data selected.\nPlease verify the correct Date before proceeding")

getROI.on_click(getStatistics)

In [21]:
#load model and predict data
def getlmodel(changed): 
        
    global ndvi, ndwi, pi, fdi, plastic_per, df

    #calculate index values
    ndvi = (B8_Matrix - B4_Matrix)/(B8_Matrix + B4_Matrix)
    ndwi = (B3_Matrix - B8_Matrix)/(B3_Matrix + B8_Matrix)
    pi = B8_Matrix/(B8_Matrix + B4_Matrix)
    fdi = B8_Matrix - (B6_Matrix + (B11_Matrix - B6_Matrix) * 1.77703)

    #prepare dataframe
    df = pd.DataFrame(data = ndvi.flatten(), columns = ["NDVI"])
    df["NDWI"] = ndwi.flatten()
    df["PI"] = pi.flatten()
    df["FDI"] = fdi.flatten()
    
    ######comment below lines to use model#####
    #get the data into X
    #X = df
    #load the model
    #rfc_model = joblib.load("./random_forest.joblib")

    #with load_model:
    #with out2:
        #clear_output()
        #print("Model Loaded")

    #predict on the test data
    #preds = rfc_model.predict(X)
    #df["Label"] = preds
    #df.loc[df["Label"] == 1, "NDVI"] = np.nan
    
    #comment if model is uncommented
    def labelling(row):
        if row["NDVI"] > -0.4 and row["NDVI"] < -0.07:
            val = "water"
        elif row["NDVI"] > 0.4 and row["NDVI"] < 0.6:
            val = "seaweed"
        elif row["NDVI"] > 0 and row["NDVI"] < 0.4:
            val = "timber"
        elif row["NDVI"] > -0.07 and row["NDVI"] < 0.02:
            val = "plastic"
        #elif row["NDVI"] < -0.1 and row["NDVI"] > -0.4:
         #   val = "foam"

        return val
    
    df["Label"] = df.apply(labelling, axis = 1)
    
    df.loc[df["Label"] == "plastic", "NDVI"] = np.nan
    df.loc[df["Label"] == "plastic", "NDWI"] = np.nan
    df.loc[df["Label"] == "plastic", "PI"] = np.nan
    df.loc[df["Label"] == "plastic", "FDI"] = np.nan
    #comment till here if model is uncommented
    
    plastic_per = round((df["NDVI"].isnull().sum()/len(df)),2)
    
    with out2:    
        clear_output()
        current_cmap = plt.cm.get_cmap()
        current_cmap.set_bad(color = 'red')
        plt.imshow(((np.array(df["NDVI"].values.tolist()).astype('float'))).reshape(40,40))
        plt.title("{:.2%} of Plastics dectected".format(plastic_per))
        plt.axis("off")
        plt.show()
        
estimate_plastics_Btn.on_click(getlmodel)

In [23]:
Map = geemap.Map(basemap = "SATELLITE")
with mapWidget:
    display(Map)

In [25]:
#create app layout
#Arrange the layout
vBox = mapWidget
vBox1 = widgets.VBox([step1, dateBox,vBox])
vBox2 = widgets.VBox([step2, getROI])

vBox2 = widgets.VBox([step2,getROI, out1,
                      step3,estimate_plastics_Btn,out2])                            

AppLayout(header = None,
          left_sidebar = vBox1,
          right_sidebar = vBox2,
          footer = None,
          height = "70%", width="150%")

AppLayout(children=(VBox(children=(Label(value='Step 1: Select the date'), DatePicker(value=None), Output(outp…